In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Ada Boost

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
ada = AdaBoostRegressor()
ada.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: ada.predict(x)[0],axis = 1)
precision = ada.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 19.43 % , error = 111158957920.4786


### Ahora que tenemos una intuicion, probamos variando los parametros

In [8]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [9]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [14]:
lista_estimators = [2,3,4,6,7,8,9]
lista_loss = ['square']

for estimator in lista_estimators:
    for loss in lista_loss:
            ada = AdaBoostRegressor(n_estimators = estimator, loss = loss)
            ada.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
            set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: ada.predict(x)[0],axis = 1)
            precision = ada.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
            error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
            res.append((estimator,loss,precision,error))
            print(estimator,' - ',loss)

2  -  square
3  -  square
4  -  square
6  -  square
7  -  square
8  -  square
9  -  square


In [15]:
for r in res:
    print ("estimator = {}, loss = {}, precision = {:.2f} % ,error = {}".format(r[0],r[1],r[2],r[3]))

estimator = 5, loss = linear, precision = 32.13 % ,error = 93630226703.13373
estimator = 5, loss = square, precision = 34.31 % ,error = 90632374291.81346
estimator = 5, loss = exponential, precision = 1.51 % ,error = 135883111824.44695
estimator = 10, loss = linear, precision = 22.75 % ,error = 106576850575.38762
estimator = 10, loss = square, precision = 32.36 % ,error = 93310837266.75972
estimator = 10, loss = exponential, precision = -9.61 % ,error = 151221086248.75476
estimator = 20, loss = linear, precision = 19.78 % ,error = 110673588583.70625
estimator = 20, loss = square, precision = 25.97 % ,error = 102136082638.34029
estimator = 20, loss = exponential, precision = -29.00 % ,error = 177962620514.4057
estimator = 30, loss = linear, precision = 15.17 % ,error = 117025175411.09717
estimator = 30, loss = square, precision = 32.69 % ,error = 92865597001.814
estimator = 30, loss = exponential, precision = -154.98 % ,error = 351764422658.5229
estimator = 50, loss = linear, precision 

In [16]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if r[2] > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = estimator = {}, loss = {}, precision = {:.2f} % ,error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3]))
print("Menor error = estimator = {}, loss = {}, precision = {:.2f} % ,error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3]))

Mayor precision = estimator = 3, loss = square, precision = 35.10 % ,error = 89531247481.83997
Menor error = estimator = 3, loss = square, precision = 35.10 % ,error = 89531247481.83997


# Elegimos estimator = 3, loss = square

In [70]:
#Probamos varias combinaciones de columnas, la de mejor resultado fue esta

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [71]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [72]:
estimator = 3
loss = 'square'

In [73]:
ada = AdaBoostRegressor(n_estimators = estimator, loss = loss)
ada.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: ada.predict(x)[0],axis = 1)
precision = ada.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 33.18 % , error = 92185282012.8887


# Calculamos los verdaderos datos

In [74]:
analizar = pd.read_csv('/home/mati/Desktop/properati_dataset_modificado.csv')

In [75]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: ada.predict(x)[0],axis = 1)

In [76]:
resultado = analizar.loc[:,['id','price_usd']]

In [77]:
resultado.to_csv('Ada_Boost_resultados.csv', index = False)